In [ ]:
import networkx as nx
import pandas as pd

def compute_directional_network_distances(edges, station2node, facilities):
    """
    计算服务区到方向性最近的前后两个服务区和充电站的路网距离。

    参数:
    edges (DataFrame): 包含路网边信息的数据框。
    station2node (DataFrame): 包含站点到节点的映射信息。
    facilities (DataFrame): 包含服务区和充电站位置信息的数据框。

    返回:
    DataFrame: 含有服务区ID和到最近的设施路网距离的数据。
    """
    # 构建有向图
    G = nx.DiGraph()
    for idx, row in edges.iterrows():
        G.add_edge(row['u'], row['v'], weight=row['length'])

    # 使用station2node映射来获取每个设施的最近节点
    facilities = facilities.merge(station2node, left_on='id', right_on='station_id', how='left')

    # 筛选出服务区
    service_areas = facilities[facilities['flag'] == '服务区']

    # 计算每个服务区到其他设施的最短路网距离
    distances = []
    for index, service_area in service_areas.iterrows():
        service_node = service_area['node_id']
        forward_paths = nx.single_source_dijkstra_path_length(G, service_node, weight='weight')
        reverse_paths = nx.single_source_dijkstra_path_length(G.reverse(), service_node, weight='weight')
        
        facility_distances = facilities.copy()
        facility_distances['forward_distance'] = facility_distances['node_id'].map(forward_paths)
        facility_distances['reverse_distance'] = facility_distances['node_id'].map(reverse_paths)
        
        # 选择前一个和后一个服务区和充电站
        forward_facilities = facility_distances.sort_values('forward_distance')
        reverse_facilities = facility_distances.sort_values('reverse_distance')

        distances.append({
            'service_area_id': service_area['id'],
            'nearest_forward_service': forward_facilities[forward_facilities['type'] == 'service_area'].iloc[0]['id'] if not forward_facilities.empty else None,
            'nearest_forward_service_distance': forward_facilities[forward_facilities['type'] == 'service_area'].iloc[0]['forward_distance'] if not forward_facilities.empty else None,
            'nearest_reverse_service': reverse_facilities[reverse_facilities['type'] == 'service_area'].iloc[0]['id'] if not reverse_facilities.empty else None,
            'nearest_reverse_service_distance': reverse_facilities[reverse_facilities['type'] == 'service_area'].iloc[0]['reverse_distance'] if not reverse_facilities.empty else None,
        })

    return pd.DataFrame(distances)

# 使用示例
edges = pd.DataFrame([...])  # 道路边信息
station2node = pd.DataFrame([...])  # 站点到节点的映射信息
facilities = pd.DataFrame([...])  # 服务区和充电站信息

results = compute_directional_network_distances(edges, station2node, facilities)
print(results)

In [ ]:
import numpy as np
import geopandas as gpd
from mgwr.gwr import GWR, GWRResults
from mgwr.sel_bw import Sel_BW
from libpysal.weights import KNN

# 假设你有一个GeoDataFrame 'gdf'，其中包含了位置坐标和一些解释变量以及因变量

# 准备数据
coords = list(zip(gdf['longitude'], gdf['latitude']))
y = gdf['dependent_var'].values.reshape((-1,1))  # 因变量
X = gdf[['independent_var1', 'independent_var2']].values  # 解释变量

# 选择带宽
bw = Sel_BW(coords, y, X, kernel='bisquare', fixed=False).search()

# 构建模型
model = GWR(coords, y, X, bw, kernel='bisquare', fixed=False)
results = model.fit()

# 查看结果
print(results.summary())
